In [1]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
import os

# ==========================================
# 1. 加载清洗后的数据
# ==========================================
print(">>> 1. 加载数据...")
# 读取上一阶段清洗好的数据
df = pd.read_csv('../data/processed/elliptic_cleaned.csv')

# 读取原始边列表 (用来构建图)
# 注意：这里我们只保留连接"清洗后存在的节点"的边
edges_raw = pd.read_csv('../data/raw/elliptic_txs_edgelist.csv')

print(f"原始节点数: {len(df)}")
print(f"原始边数: {len(edges_raw)}")

# ==========================================
# 2. 节点重映射 (Re-indexing) [关键步骤]
# ==========================================
# PyG 要求节点索引必须是 0 到 N-1 的连续整数。
# 但原始数据的 txId 是随机的大整数（如 355110272）。
# 我们需要建立一个映射： raw_id -> new_id (0, 1, 2...)

print("\n>>> 2. 执行节点重映射...")

# 1. 给每个节点分配一个新的索引 (0, 1, 2, ..., N-1)
# 重置索引，让 DataFrame 的 index 变成 0..N-1
df = df.reset_index(drop=True)

# 创建映射字典: {原始txId : 新索引Index}
id_map = {row['txId']: i for i, row in df.iterrows()}

# 2. 过滤边列表
# 只保留两端都在我们 df 里的边
edges_mask = edges_raw['txId1'].isin(df['txId']) & edges_raw['txId2'].isin(df['txId'])
edges_clean = edges_raw[edges_mask].copy()

# 3. 将边列表里的 txId 替换为新索引
edges_clean['source_node'] = edges_clean['txId1'].map(id_map)
edges_clean['target_node'] = edges_clean['txId2'].map(id_map)

print(f"重映射完成。有效边数量: {len(edges_clean)}")

# ==========================================
# 🔍 调试与修复：检查列名
# ==========================================
print("当前的所有列名:", df.columns.tolist()[:10], "...") # 只打印前10个看看

# 1. 尝试自动修复常见的列名变体
possible_names = ['Time step', 'time_step', 'timestep', '1', 1] # '1' 是原始数据中的常见列名

found_col = None
for name in possible_names:
    if name in df.columns:
        found_col = name
        break

if found_col:
    print(f"✅ 找到了时间列，当前名称为: '{found_col}'")
    if found_col != 'Time step':
        print(f"🔄 正在将其重命名为 'Time step'...")
        df.rename(columns={found_col: 'Time step'}, inplace=True)
else:
    # 如果都没找到，可能在某个位置（通常是第二列）
    # 也就是 features.csv 的第2列 (索引为1)
    print("⚠️ 未按名称找到时间列，尝试按位置推断...")
    # 假设 'class' 和 'txId' 存在，剩下的第一列可能是时间？
    # 或者直接看第2列（如果第1列是txId）
    print("第2列的数据样例:", df.iloc[:, 1].head().values)

    # 强制重命名第2列 (非常规手段，慎用，请先确认上面的打印结果)
    # col_name_at_idx_1 = df.columns[1]
    # df.rename(columns={col_name_at_idx_1: 'Time step'}, inplace=True)

print("当前列名检查完毕。")

# ==========================================
# 3. 时间切分 (Temporal Split)
# ==========================================
# 根据 Elliptic 论文标准：
# 训练集: Time step 1 - 34
# 测试集: Time step 35 - 49

print("\n>>> 3. 创建时间切分掩码...")

# 确保 class 列是整数 (1=Illicit, 2=Licit)
# 这里我们把它们转为 0 和 1，符合 PyTorch 的二分类习惯
# 通常定义: 1=Illicit (正样本), 0=Licit (负样本)
class_map = {1: 1, 2: 0, '1': 1, '2': 0}
df['label'] = df['class'].map(class_map)

# 定义切分点
split_timestep = 34

# 创建列来标记数据集归属 (方便基准模型使用)
df['split'] = 'train'
df.loc[df['Time step'] > split_timestep, 'split'] = 'test'

print("训练集/测试集分布:")
print(df['split'].value_counts())

# ==========================================
# 4. 构建 PyG Data 对象 (修复版)
# ==========================================
print(">>> 4. 构建 PyG Data 对象 (Safe Mode)...")

# -------------------------------------------------------
# A. 特征矩阵 (x) - 强力清洗逻辑
# -------------------------------------------------------

# 1. 首先，只选择数值类型的列 (排除所有字符串列)
numeric_df = df.select_dtypes(include=[np.number])

# 2. 定义必须要剔除的非特征列 (ID, 时间, 标签)
# 注意：我们要剔除原始的 'class' (如果是数字) 和我们生成的 'label'
exclude_cols = ['txId', 'Time step', 'class', 'label']

# 3. 计算最终的特征列名
# (在 numeric_df 的列中，去除 exclude_cols)
feature_cols = [c for c in numeric_df.columns if c not in exclude_cols]

# --- [调试] 检查一下是不是有列被误删或遗漏 ---
print(f"原始 DataFrame 列数: {len(df.columns)}")
print(f"数值型 DataFrame 列数: {len(numeric_df.columns)}")
print(f"最终特征数量: {len(feature_cols)}")

# 如果特征数异常少 (比如为0)，报错提醒
if len(feature_cols) == 0:
    raise ValueError("错误：找不到任何特征列！请检查 'exclude_cols' 设置或数据类型。")

# --- [关键步骤] 转换为 Tensor ---
# values 可能会因为内存不连续报错，加上 .copy()
x = torch.tensor(numeric_df[feature_cols].values, dtype=torch.float)

print(f"✅ 特征矩阵 x 构建成功! Shape: {x.shape}")

# -------------------------------------------------------
# B. 边索引 (edge_index)
# -------------------------------------------------------
# 确保边索引也是 Long 类型
edge_index = torch.tensor(edges_clean[['source_node', 'target_node']].values, dtype=torch.long)
edge_index = edge_index.t().contiguous() # 转置为 shape [2, E]

# -------------------------------------------------------
# C. 标签 (y)
# -------------------------------------------------------
y = torch.tensor(df['label'].values, dtype=torch.long)

# -------------------------------------------------------
# D. 掩码 (Masks)
# -------------------------------------------------------
train_mask = torch.tensor(df['Time step'] <= split_timestep, dtype=torch.bool)
test_mask = torch.tensor(df['Time step'] > split_timestep, dtype=torch.bool)

# -------------------------------------------------------
# E. 组装 Data 对象
# -------------------------------------------------------
data = Data(x=x, edge_index=edge_index, y=y)
data.train_mask = train_mask
data.test_mask = test_mask

print("-" * 30)
print("PyG 数据对象信息:")
print(data)
print(f"特征维度: {data.num_features}")
print(f"是否为无向图: {data.is_undirected()}")
print("-" * 30)

# ==========================================
# 5. 保存文件 (顺便在这里执行了)
# ==========================================
import os
output_dir = '../data/processed/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 保存 .pt 文件
torch.save(data, os.path.join(output_dir, 'elliptic_pyg.pt'))
print(f"✅ GNN 数据已保存: {os.path.join(output_dir, 'elliptic_pyg.pt')}")

# 保存 .csv 文件 (包含特征和split标记)
# 重新把 'split' 列加回 numeric_df 方便保存 (因为刚才只选了数值)
numeric_df['split'] = df['split']
numeric_df['txId'] = df['txId'] # 加上ID方便索引
numeric_df['label'] = df['label'] # 加上label

# 只保存需要的列
save_cols = ['txId', 'label', 'split'] + feature_cols
output_csv_path = os.path.join(output_dir, 'elliptic_benchmark.csv')
numeric_df[save_cols].to_csv(output_csv_path, index=False)
print(f"✅ 基准模型数据已保存: {output_csv_path}")


>>> 1. 加载数据...
原始节点数: 46564
原始边数: 234355

>>> 2. 执行节点重映射...
重映射完成。有效边数量: 36624
当前的所有列名: ['txId', 'time_step', 'feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6', 'feat_7'] ...
✅ 找到了时间列，当前名称为: 'time_step'
🔄 正在将其重命名为 'Time step'...
当前列名检查完毕。

>>> 3. 创建时间切分掩码...
训练集/测试集分布:
split
train    29894
test     16670
Name: count, dtype: int64
>>> 4. 构建 PyG Data 对象 (Safe Mode)...
原始 DataFrame 列数: 171
数值型 DataFrame 列数: 169
最终特征数量: 165
✅ 特征矩阵 x 构建成功! Shape: torch.Size([46564, 165])
------------------------------
PyG 数据对象信息:
Data(x=[46564, 165], edge_index=[2, 36624], y=[46564], train_mask=[46564], test_mask=[46564])
特征维度: 165
是否为无向图: False
------------------------------
✅ GNN 数据已保存: ../data/processed/elliptic_pyg.pt
✅ 基准模型数据已保存: ../data/processed/elliptic_benchmark.csv
